# Libraries

In [20]:
from tkinter import *
from tkinter import ttk
import tkinter.font as ft
#from tkinter import tkk
from tkinter import filedialog
import tkinter.messagebox as tkm
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import numpy as np
#import pyodbc
import pandas as pd
import seaborn as sns
from PIL import ImageTk, Image
from scipy.stats import norm
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
#import dropbox 
from scipy import stats
from reportlab.lib.colors import white,black,HexColor,Color
#from reportlab.platyplus import SimpleDocTemplate,Table,TableStyle
from reportlab import *
from reportlab.lib import colors
import os
from reportlab.platypus import Table,TableStyle,SimpleDocTemplate
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})

# TKINTER INTERFACE

In [14]:
class EDAwindow():

    
    def __init__(self,window):
        self.x1=False
        self.x2=False
        self.window=window
        self.window.geometry("500x500+400+70")
        self.window.configure(bg='#DAEDFB')
        self.button1=Button(self.window,text='EXIT',command=self.window.destroy,bg='red',fg='white',relief='raised',width=8,font=ft.Font(weight='bold')).place(x=385,y=425)
        self.button2=Button(self.window,text='RUN',bg='green',fg='white',relief='raised',width=8,font=ft.Font(weight='bold')).place(x=10,y=425)
        self.window.title("Exploratory Data Anaysis Tool")
        self.ComboBox1=ttk.Combobox(values=('Query from Database','Excel or CSV'))
        self.ComboBox1.place(x=5,y=145)
        self.ComboBox2=ttk.Combobox(values=('Analytics','Regression','Classification'))
        self.ComboBox2.place(x=5,y=280)
        data_label=Label(self.window,text='Select type of data source',justify='center',bg='#DAEDFB')
        data_label.place(x=5,y=115)
        report_label=Label(self.window,text='Select report type',justify='center',bg='#DAEDFB')
        report_label.place(x=5,y=250)
        
        label_title1=Label(self.window,text='Exploratory',justify='center',fg='#830137',bg='#DAEDFB',font=("Courier-Bold", 14))
        label_title1.place(x=5,y=5)
        
        label_title2=Label(self.window,text='Data',justify='center',fg='#830137',bg='#DAEDFB',font=("Courier-Bold", 14))
        label_title2.place(x=5,y=30)
        
        label_title3=Label(self.window,text='Analysis',justify='center',fg='#830137',bg='#DAEDFB',font=("Courier-Bold", 14))
        label_title3.place(x=5,y=55)
        
        label_file=Label(self.window,text='Copy Output path',justify='left',fg='black',bg='#DAEDFB')
        label_file.place(x=5,y=350)
        entry_file=Entry(self.window)
        entry_file.place(x=5,y=375)
        
        label_output=Label(self.window,text='Insert output filename',justify='left',fg='black',bg='#DAEDFB')
        label_output.place(x=365,y=350)
        entry_output=Entry(self.window)
        entry_output.place(x=365,y=375)

        def file_import():
            file=filedialog.askopenfile()
            L5=Label(self.window,text=file.name,bg='#DAEDFB').place(x=5,y=205)
        
        def check_box():
            if self.ComboBox2.get()=='Classification':
                L4=Label(self.window,text='Insert name of response variable',bg='#DAEDFB')
                L4.place(x=5,y=310)
                e1=Entry(self.window)
                e1.place(x=5,y=330)
            

        def query_selection():
            global L1,L2,L3,e1,e2,e3,L4,B2
            if self.ComboBox1.get()=='Query from Database':
                if self.x1==True :
                    L4.destroy()
                    B2.destroy()
                    L4.destroy()

                L1=Label(self.window, text='User',justify='center')
                L1.place(x=5,y=175)
                L2=Label(self.window,text='Password')
                L2.place(x=5,y=195)
                L3=Label(self.window,text='Query')
                L3.place(x=5,y=215)
                e1=Entry(self.window)
                e1.place(x=65,y=175)
                e2=Entry(self.window)
                e2.place(x=65,y=195)
                e3=Entry(self.window)
                e3.place(x=65,y=215)
                self.x2=True

            elif self.ComboBox1.get()=='Excel or CSV':
                if self.x2==True:
                    L1.destroy()
                    L2.destroy()
                    L3.destroy()
                    e1.destroy()
                    e2.destroy()
                    e3.destroy()
                L4=Label(self.window,text='Import your file',bg='#DAEDFB')
                L4.place(x=5,y=175)
                B2=Button(self.window,text='Import',command=file_import,bg='#CC0000',fg='white')
                B2.place(x=430,y=175)
                self.x1=True

            
        self.button3=Button(self.window,text='Validate',command=query_selection,bg='#CC0000',fg='white').place(x=430,y=145)
        self.button4=Button(self.window,text='Validate',command=check_box,bg='#CC0000',fg='white').place(x=430,y=280)
        
     
            
        
    


eda_window=Tk()
c=Canvas(eda_window,width=500,height=500,bg='#DAEDFB')
c.create_rectangle(0, 480, 500, 500,outline="red", fill="#CC0000")
c.pack()
EDAwindow(eda_window)
eda_window.mainloop()


In [15]:
#HARDCODED FOR NOW

dataframe=pd.read_csv('2.12_Health_systems.csv')
targetcolumn='Health_exp_pct_GDP_2016'


# MAIN BODY

In [23]:
def EDA(dataframe):
    
    def df_to_formattedlist(dataframe):
        list_df=[]
        colnames=dataframe.columns.tolist()
        list_df.insert(0,colnames)
        for i in range(0,len(dataframe)):
            to_insert=dataframe.iloc[i,:][0:12].tolist()
            list_df.insert(i+1,to_insert)
        return list_df
    
    def text_format(textobject,x,y,lettersize,color=black):
        textobject.setTextOrigin(x,y)
        textobject.setFont('Helvetica',lettersize)
        textobject.setFillColor(color)
        
    def document_properties(mycanvas):
        
        mycanvas.setFillColor(HexColor('#CC0000'))
        mycanvas.rect(0,792,612,-35,stroke=0,fill=1)
        mycanvas.setFillColor(HexColor('#DAEDFA'))
        mycanvas.rect(0,20,612,735,stroke=0,fill=1)
        mycanvas.setFillColor(white)
        mycanvas.setStrokeColor(white)
        mycanvas.setFont('Helvetica-Bold',14)
        mycanvas.drawCentredString(325,771,'Exploratory Data Analysis Report')
        textobject=mycanvas.beginText()
        text_format(textobject,0,745,8,color=black)
        textobject.textLines('''Exploratory data analysis is an approach to quickly summarize the main characteristics of a dataset, comnbining visual and analytic methods.Through this report you will be
                            able to check the main characteristics of your dataset, helping you to obtain further insights from your data''')
        mycanvas.drawText(textobject)
        mycanvas.setFillColor(HexColor('#CC0000'))
        mycanvas.rect(0,0,612,20,stroke=0,fill=1)
        mycanvas.setFont('Helvetica-Bold',4)
        mycanvas.setFillColor(white)
        mycanvas.drawString(2,15,'Your company')
        mycanvas.drawString(2,15,'October 2021')
        
    def page_properties(mycanvas):
        mycanvas.setFillColor(HexColor('#DAEDFA'))
        mycanvas.rect(0,0,612,792,stroke=0,fill=1)
        mycanvas.setFillColor(HexColor('#CC0000'))
        mycanvas.rect(0,0,612,20,stroke=0,fill=1)
        mycanvas.setFont('Helvetica-Bold',4)
        mycanvas.setFillColor(white)
        mycanvas.drawString(2,15,'Your company')
        mycanvas.drawString(2,5,'October 2021')
    
    def tablestyle(table):
        table.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                                   ('TOPPADDING',(0,0),(-1,-1),8),
                                   ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                                   ('FONT',(0,0),(-1,0),'Courier-BoldOblique'),
                                   ('TEXTCOLOR',(0,0),(-1,0),colors.Color(1,1,1)),
                                   ('FONT',(0,1),(0,-1),'Times-Bold'),
                                   ('GRID',(0,0),(-1,-1),0.5,colors.black),
                                   ('BACKGROUND',(0,0),(-1,0),colors.Color(0.4375,0.1835,0.539)),
                                   ('FONTSIZE',(0,0),(-1,-1),5,5)]))
        
    def chart_insertion(data,init_x,init_y,chart):
        for i in range(0,len(data.columns)):
            if init_x<=width-120:
                mycanvas.drawImage(chart+str(i)+'.png',init_x,init_y,6.4*30,4.1*30)
                init_x+=195
                
            elif init_y>=195:
                init_x=15
                init_y-=135
                mycanvas.drawImage(chart+str(i)+'.png',init_x,init_y,6.4*30,4.1*30)
                init_x+=195
                
            elif init_y<=60:
                mycanvas.showPage()
                page_properties(mycanvas)
                init_x=15
                init_y=650
                mycanvas.drawImage(chart+str(i)+'.png',init_x,init_y,6.4*30,4.1*30)
                
            else:
                continue
            os.remove(chart+str(i)+'.png')
            
    width,height=letter
    mycanvas=canvas.Canvas('pdftest.pdf',pagesize=letter)
    document_properties(mycanvas)
    
    data=dataframe._get_numeric_data()
    
    for i in range(0,len(data.columns)):
        col=str(data.columns[i-1])
        mean,std=norm.fit(data[col].dropna())
        fig,ax=plt.subplots()
        ax.set_facecolor('#DAEDFA')
        ax.patch.set_facecolor('#DAEDFA')
        ax.hist(data[col].dropna(),60,facecolor='#CC0000',rwidth=0.8)
        ax.set_title(data.columns[i-1]+ ' ' + 'mean=%.1f,std=%.1f' %(mean,std))
        ax.grid(False)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        fig.savefig('histogram'+str(i)+'.png',facecolor='#DAEDFA')
        plt.close()
        
        
        marker_style=dict(markerfacecolor='#AF2349',marker='h',color='#917C93')
        fig,ax=plt.subplots()
        ax.boxplot(data[col].dropna(),patch_artist=True,flierprops=marker_style,medianprops=dict(color='#840037'),boxprops=dict(color='#CC0000',facecolor='#CC0000'))
        ax.set_facecolor('#DAEDFA')
        ax.patch.set_facecolor('#DAEDFA')
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        plt.title('Boxplot of'+' '+col)
        ax.grid(False)
        plt.title('Boxplot of'+' '+col)
        fig.savefig('boxplot'+str(i)+'.png',facecolor='#DAEDFA')
        plt.close()
        
        
  
            
            

        
    cat_vars=dataframe.select_dtypes(include=['object'])
    color_plt=[(0.56,0.113,0.192),(0.56,0.113,0.192),(0.56,0.113,0.192),(0.56,0.113,0.192),(0.56,0.113,0.192)]
        
    for i in range(0,len(cat_vars.columns)):
        col=str(cat_vars.columns[i])
            
        if len(cat_vars[col].value_counts())>=5:
            plot_cats=cat_vars[col].value_counts().nlargest(5)
            fig,ax=plt.subplots()
            plot_cats.plot.bar(title=col,rot=0,color=color_plt)
            ax.set_facecolor('#FFFFFF')
            ax.patch.set_facecolor('#DAEDFA')
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
            fig.savefig('univariate'+str(i)+'.png',facecolor='#DAEDFA')
            plt.close()
        else:
            plot_cats=cat_vars[col].value_counts()
            fig,ax=plt.subplots()
            plot_cats.plot.bar(title=col,rot=0,colors=color_plt)
            ax.set_facecolor('#FFFFFF')
            ax.patch.set_facecolor('#DAEDFA')
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
            fig.savefig('univariate'+str(i)+'.png',facecolor='#DAEDFA')
            plt.close()
                
                
    x1=pd.DataFrame(index=['Total Observartions','Total Columns','Total Nulls'],data={'Metrics':[str(len(dataframe)),str(len(dataframe.columns)),dataframe.isnull().sum().sum()]})
    x1=x1.reset_index()
    x1.columns.values[0]='Variable'
    x2=df_to_formattedlist(x1)
        
    summary_numeric=dataframe.describe().transpose().applymap(str)
    summary_numeric3=summary_numeric.applymap(lambda x:x.split('.')[0]).reset_index()
    summary_numeric3=summary_numeric3.applymap(lambda x: x[0:12])
    summary_numeric3.columns.values[0]='Variable'
    x3=df_to_formattedlist(summary_numeric3)
    summary_object=dataframe.describe(include=[np.object]).transpose().applymap(str)
    summary_object2=summary_object.applymap(lambda x:x.split('.')[0]).reset_index()
    summary_object3=summary_object2.applymap(lambda x: x[0:12])
    summary_object3.columns.values[0]='Variable'
    x4=df_to_formattedlist(summary_object3)
        
    tableb,table1,table2=x2,x3,x4
    textobject0=mycanvas.beginText()
    text_format(textobject0,275,685,12,color=black)
    textobject0.textLines('''Dataset Description''')
    mycanvas.drawText(textobject0)
    t0=Table(tableb,len(tableb[0])*[46],len(tableb)*[20])
    tablestyle(t0)
    t0.wrapOn(mycanvas,9,9)
    t0.drawOn(mycanvas,282,590)
        
    textobject=mycanvas.beginText()
    text_format(textobject,250,565,12,color=black)
    textobject.textLines('''Continous Variables Description''')
    mycanvas.drawText(textobject)
    set_separator=len(table1)
        
    if set_separator<25:
        t=Table(table1,colWidths=[45],rowHeights=20)
        tablestyle(t)
        t.wrapOn(mycanvas,9,9)
        x=540-(len(table1)*20)
        t.drawOn(mycanvas,115,x)
        textobject2=mycanvas.beginText()
        text_format(textobject2,252,x-20,12)
        textobject2.textLines('''Categorical Variables Description''')
        mycanvas.drawText(textobject2)
            
    else: 
        t=Table(table1[:25],colWidths=[45],rowHeights=20)
        tablestyle(t)
        t.wrapOn(mycanvas,9,9)
        x=540-(len(table1)*20)
        t.drawOn(mycanvas,115,x)
        textobject5=mycanvas.beginText()
        text_format(textobject2,252,x-20,12)
        textobject5.textLines('''Categorical Variables Description''')
        mycanvas.drawText(textobject5)
        mycanvas.showPage()
        init=24
        title=['Measure','Count','Mean','Std','Min','Max','25%','50%','75%','Max']
        while(len(table1)-init)/35>=1:
            page_properties(mycanvas)
            t=Table(table1[init:init+35],colWidths=[45],rowHeights=20)
            tablestyle(t)
            x=750-(35*20)
            t0.wrapOn(mycanvas,9,9)
            t0.drawOn(mycanvas,115,x)
            mycanvas.showPage()
            init=init+35
                
        tableins=table1[init+1:]
        tableins.insert(0,title)
        t=Table(table1,colWidths=[45],rowHeights=20)
        tablestyle(t)
        x=750-(len(table1[init:])*20)
        t0.wrapOn(mycanvas,9,9)
        t0.drawOn(mycanvas,115,x)
            
            
    t2=Table(table2,colWidths=[45],rowHeights=20)
    tablestyle(t2)
    t2.wrapOn(mycanvas,9,9)
    x2=x-35-(len(table2)*20)
    t2.drawOn(mycanvas,220,x2)
    mycanvas.showPage()
    page_properties(mycanvas)
    data2=dataframe._get_numeric_data()
    init_x=15
    init_y=600
    textobject=mycanvas.beginText()
    text_format(textobject,0,745,8)
    textobject.textLines(''' Histograms provide a visual interpretation of numerical data by indicating the number of data points that lie within a range of values. These ranges are called classes or bins.
                                 In this case data is represented with 60 bins''')
            
    mycanvas.drawText(textobject)
            
    textobject2=mycanvas.beginText()
    text_format(textobject2,290,770,15,color=HexColor('#B31E30'))
    textobject2.textLines('Histogram')
    mycanvas.drawText(textobject2)
    chart_insertion(data=data,init_x=init_x,init_y=init_y,chart='Histogram')
    mycanvas.showPage()
    page_properties(mycanvas)
            
    init_x=15
    init_y=600
    textobject=mycanvas.beginText()
    text_format(textobject,0,745,8)
    textobject.textLines(''' Boxplot is a method for graphically depicting groups of numerical values through their quantiles. Line in orange represents the median, while highlighted values (red) correspond
                                   to outliers''')
    mycanvas.drawText(textobject)
    textobject2=mycanvas.beginText()
    text_format(textobject2,290,770,15,color=HexColor('#B31E30'))
    textobject2.textLines('Boxplot')
    mycanvas.drawText(textobject2)
    chart_insertion(data=data,init_x=init_x,init_y=init_y,chart='boxplot')
    mycanvas.showPage()
    page_properties(mycanvas)
            
            
    data2=dataframe.select_dtypes(include=['object'])
    init_x=15
    init_y=600
    textobject=mycanvas.beginText()
    text_format(textobject,0,745,8)
    textobject.textLines(''' Barchar represent categorical data with rectangular bars with heights and lenghts proportional to the values that they represent . Here we only plot the top 5 categories in
                                 terms of ocurrencies, to check more details about the number of levels, refer to the categorical variables table''')
    mycanvas.drawText(textobject)
            
                                
    textobject2=mycanvas.beginText()
    text_format(textobject2,290,770,15,color=HexColor('#B31E30'))
    textobject2.textLines('Barchart')
    mycanvas.drawText(textobject2)
    chart_insertion(data=data2,init_x=init_x,init_y=init_y,chart='univariate')
    #mycanvas.showPage()
    page_properties(mycanvas)
            
            
    
                    
    #mycanvas.showPage()
    mycanvas.save()
            
            
            
            
            
            
            
            
                
                
                
            
            
            
            
        
                
                
                
        
    
                                   
        
            
            
        
            
        
        
        

# Dropbox

In [10]:
import dropbox
file_from = 'EDA_Tool.ipynb'
file_to = '/Projects/EDA_Tool.ipynb'
dbx = dropbox.Dropbox('HQH29lb-8P0AAAAAAAAAAfF6cgci3D5zTJMADSDthQKheL6MJBLjXwUwIe0a9bSD')
dbx.files_upload(open(file_from, 'rb').read(), file_to,mode=dropbox.files.WriteMode.overwrite)

FileMetadata(name='EDA_Tool.ipynb', id='id:sBcB4xgU7FAAAAAAAAAAMA', client_modified=datetime.datetime(2020, 10, 26, 18, 36, 56), server_modified=datetime.datetime(2020, 10, 26, 18, 36, 57), rev='015b2973711104d00000001f4129670', size=58087, path_lower='/projects/eda_tool.ipynb', path_display='/Projects/EDA_Tool.ipynb', parent_shared_folder_id=None, media_info=None, symlink_info=None, sharing_info=None, is_downloadable=True, export_info=None, property_groups=None, has_explicit_shared_members=None, content_hash='5b4c50e65baecd88d3c974fae5b5e3fbd91ef8013fdb1fb31e3f3a370bc99244', file_lock_info=None)

# TOOL CODE PREPARATION


In [24]:
class EDAwindow():

    
    def __init__(self,window):
        self.x1=False
        self.x2=False
        
        def run():
            global combobox
            combobox=self.ComboBox2.get()
            EDA(dataframe)
            tkm.showinfo(title='Report Completed', message='Report created succesfully')
            
        self.window=window
        self.window.geometry("500x500+400+70")
        self.window.configure(bg='#DAEDFB')
        self.button1=Button(self.window,text='EXIT',command=self.window.destroy,bg='red',fg='white',relief='raised',width=8,font=ft.Font(weight='bold')).place(x=385,y=425)
        self.button2=Button(self.window,text='RUN',command=run,bg='green',fg='white',relief='raised',width=8,font=ft.Font(weight='bold')).place(x=10,y=425)
        self.window.title("Exploratory Data Anaysis Tool")
        self.ComboBox1=ttk.Combobox(values=('Query from Database','Excel or CSV'))
        self.ComboBox1.place(x=5,y=145)
        self.ComboBox2=ttk.Combobox(values=('Analytics','Regression','Classification'))
        self.ComboBox2.place(x=5,y=280)
        data_label=Label(self.window,text='Select type of data source',justify='center',bg='#DAEDFB')
        data_label.place(x=5,y=115)
        report_label=Label(self.window,text='Select report type',justify='center',bg='#DAEDFB')
        report_label.place(x=5,y=250)
        
        label_title1=Label(self.window,text='Exploratory',justify='center',fg='#830137',bg='#DAEDFB',font=("Courier-Bold", 14))
        label_title1.place(x=5,y=5)
        
        label_title2=Label(self.window,text='Data',justify='center',fg='#830137',bg='#DAEDFB',font=("Courier-Bold", 14))
        label_title2.place(x=5,y=30)
        
        label_title3=Label(self.window,text='Analysis',justify='center',fg='#830137',bg='#DAEDFB',font=("Courier-Bold", 14))
        label_title3.place(x=5,y=55)
        
        label_file=Label(self.window,text='Copy Output path',justify='left',fg='black',bg='#DAEDFB')
        label_file.place(x=5,y=350)
        global entry_file
        entry_file=Entry(self.window)
        entry_file.place(x=5,y=375)
        
        label_output=Label(self.window,text='Insert output filename',justify='left',fg='black',bg='#DAEDFB')
        label_output.place(x=365,y=350)
        global entry_output
        entry_output=Entry(self.window)
        entry_output.place(x=365,y=375)

        def file_import():
            file=filedialog.askopenfile()
            L5=Label(self.window,text=file.name,bg='#DAEDFB').place(x=5,y=205)
            global dataframe
            dataframe=pd.read_csv(file.name)
        
        def check_box():
            if self.ComboBox2.get()=='Classification':
                L4=Label(self.window,text='Insert name of response variable',bg='#DAEDFB')
                L4.place(x=5,y=310)
                global e1
                e1=Entry(self.window)
                e1.place(x=5,y=330)
            

        def query_selection():
            global L1,L2,L3,e1,e2,e3,L4,B2
            if self.ComboBox1.get()=='Query from Database':
                if self.x1==True :
                    L4.destroy()
                    B2.destroy()
                    L4.destroy()

                L1=Label(self.window, text='User',justify='center')
                L1.place(x=5,y=175)
                L2=Label(self.window,text='Password')
                L2.place(x=5,y=195)
                L3=Label(self.window,text='Query')
                L3.place(x=5,y=215)
                e1=Entry(self.window)
                e1.place(x=65,y=175)
                e2=Entry(self.window,show='x')
                e2.place(x=65,y=195)
                e3=Entry(self.window)
                e3.place(x=65,y=215)
                self.x2=True

            elif self.ComboBox1.get()=='Excel or CSV':
                if self.x2==True:
                    L1.destroy()
                    L2.destroy()
                    L3.destroy()
                    e1.destroy()
                    e2.destroy()
                    e3.destroy()
                L4=Label(self.window,text='Import your file',bg='#DAEDFB')
                L4.place(x=5,y=175)
                B2=Button(self.window,text='Import',command=file_import,bg='#CC0000',fg='white')
                B2.place(x=430,y=175)
                self.x1=True

            
        self.button3=Button(self.window,text='Validate',command=query_selection,bg='#CC0000',fg='white').place(x=430,y=145)
        self.button4=Button(self.window,text='Validate',command=check_box,bg='#CC0000',fg='white').place(x=430,y=280)
        


def EDA(dataframe):
    
    def df_to_formattedlist(dataframe):
        list_df=[]
        colnames=dataframe.columns.tolist()
        list_df.insert(0,colnames)
        for i in range(0,len(dataframe)):
            to_insert=dataframe.iloc[i,:][0:12].tolist()
            list_df.insert(i+1,to_insert)
        return list_df
    def df_to_formattedlist_biv(summary_df,original_df):
        list_df=[]
        colnames=original_df._get_numeric_data().columns.tolist()
        first_row=['Variable','0','1','0','1','0','1','0','1','0','1','0','1']
        list_df.insert(0,first_row)
        lista=[]
        j=0
        for i in range(0,len(colnames)-1):
            df=summary_df.iloc[i*6:i*6+6].values
            for p in range(0,6):
                for k in range(0,2):
                    lista.insert(j,df[p][k])
                    j+=1
            lista.insert(0,colnames[i][0:12])
            list_df.insert(i+1,lista)
            lista=[]
        return list_df
    
    def text_format(textobject,x,y,lettersize,color=black):
        textobject.setTextOrigin(x,y)
        textobject.setFont('Helvetica',lettersize)
        textobject.setFillColor(color)
        
    def stringpath_replace(txt):
        text_replaced=txt.replace("/", "//")
        return text_replaced
        
    def document_properties(mycanvas):
        
        mycanvas.setFillColor(HexColor('#CC0000'))
        mycanvas.rect(0,792,612,-35,stroke=0,fill=1)
        mycanvas.setFillColor(HexColor('#DAEDFA'))
        mycanvas.rect(0,20,612,735,stroke=0,fill=1)
        mycanvas.setFillColor(white)
        mycanvas.setStrokeColor(white)
        mycanvas.setFont('Helvetica-Bold',14)
        mycanvas.drawCentredString(325,771,'Exploratory Data Analysis Report')
        textobject=mycanvas.beginText()
        text_format(textobject,0,745,8,color=black)
        textobject.textLines('''Exploratory data analysis is an approach to quickly summarize the main characteristics of a dataset, comnbining visual and analytic methods.Through this report you will be
                            able to check the main characteristics of your dataset, helping you to obtain further insights from your data''')
        mycanvas.drawText(textobject)
        mycanvas.setFillColor(HexColor('#CC0000'))
        mycanvas.rect(0,0,612,20,stroke=0,fill=1)
        mycanvas.setFont('Helvetica-Bold',4)
        mycanvas.setFillColor(white)
        mycanvas.drawString(2,15,'Your company')
        mycanvas.drawString(2,15,'October 2021')
        
    def page_properties(mycanvas):
        mycanvas.setFillColor(HexColor('#DAEDFA'))
        mycanvas.rect(0,0,612,792,stroke=0,fill=1)
        mycanvas.setFillColor(HexColor('#CC0000'))
        mycanvas.rect(0,0,612,20,stroke=0,fill=1)
        mycanvas.setFont('Helvetica-Bold',4)
        mycanvas.setFillColor(white)
        mycanvas.drawString(2,15,'Your company')
        mycanvas.drawString(2,5,'October 2021')
    
    def tablestyle(table,fontsize=5):
        table.setStyle(TableStyle([('ALIGN',(0,0),(-1,-1),'CENTER'),
                                   ('TOPPADDING',(0,0),(-1,-1),8),
                                   ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                                   ('FONT',(0,0),(-1,0),'Courier-BoldOblique'),
                                   ('TEXTCOLOR',(0,0),(-1,0),colors.Color(1,1,1)),
                                   ('FONT',(0,1),(0,-1),'Times-Bold'),
                                   ('GRID',(0,0),(-1,-1),0.5,colors.black),
                                   ('BACKGROUND',(0,0),(-1,0),colors.Color(0.4375,0.1835,0.539)),
                                   ('FONTSIZE',(0,0),(-1,-1),fontsize),
                                   ('FONTSIZE',(0,0),(-1,0),7)]))
        
        
    def biv_densityplot(dataframe,target_variable):
        colors = ["#CC0000", "#830137"]
        sns.set(rc={'axes.facecolor':'#DAEDFA', 'figure.facecolor':'#DAEDFA','axes.grid' : False})
        sns.set_palette(sns.color_palette(colors))
        num_data=dataframe._get_numeric_data()
        num_data[target_variable] = dataframe[target_variable].apply(str)
        for i in range(0,len(num_data.columns)):
            current_col=num_data.columns[i]
            if current_col!=target_variable:
                fig,ax=plt.subplots()
                ax.spines['right'].set_visible(False)
                ax.spines['top'].set_visible(False)
                sns.kdeplot(data=num_data,x=current_col,hue=target_variable,fill=True,shade=True,multiple='stack')
                plt.xlabel(current_col)
                ax.set_ylabel('') 
                fig.savefig('densitybiv'+str(i)+'.png',facecolor='#DAEDFA')
                plt.close()
                
    def biv_boxplot(dataframe,target_variable):
        colors = ["#CC0000", "#830137"]
        sns.set(rc={'axes.facecolor':'#DAEDFA', 'figure.facecolor':'#DAEDFA','axes.grid' : False})
        sns.set_palette(sns.color_palette(colors))
        dataframe=dataframe.select_dtypes(exclude="object")
        dataframe[target_variable]=dataframe[target_variable].copy()
        for i in range(0,len(dataframe.columns)):
            current_col=dataframe.columns[i]
            if current_col!=target_variable:
                fig,ax=plt.subplots()
                sns.boxplot(x=dataframe[target_variable],y=dataframe[current_col])
                plt.title("Bivariate Histogram")
                plt.xlabel(target_variable)
                fig.savefig('boxplotbiv'+str(i)+'.png',facecolor='#DAEDFA')
                plt.close()
                

    def biv_barchart(dataframe,target_variable):
        colors = ["#CC0000", "#830137"]
        sns.set(rc={'axes.facecolor':'#DAEDFA', 'figure.facecolor':'#DAEDFA','axes.grid' : False})
        sns.set_palette(sns.color_palette(colors))
        obj_data=dataframe.select_dtypes(include="object")
        obj_data[target_variable] = dataframe[target_variable].apply(str).copy()
        for i in range(0,len(obj_data.columns)):
            current_col=obj_data.columns[i]
            if current_col!=target_variable:
                fig,ax=plt.subplots()
                ax.spines['right'].set_visible(False)
                ax.spines['top'].set_visible(False)
                sns.countplot(data=obj_data,x=current_col,hue=target_variable,fill=True,order=obj_data[current_col].value_counts().iloc[:5].index)
                plt.xlabel(current_col)
                fig.savefig('barchartbiv'+str(i)+'.png',facecolor='#DAEDFA')
                plt.close()
                
        
    def chart_insertion(data,init_x,init_y,chart):
        for i in range(0,len(data.columns)):
            if init_x<=width-120:
                mycanvas.drawImage(chart+str(i)+'.png',init_x,init_y,6.4*30,4.1*30)
                init_x+=195
                
            elif init_y>=195:
                init_x=15
                init_y-=135
                mycanvas.drawImage(chart+str(i)+'.png',init_x,init_y,6.4*30,4.1*30)
                init_x+=195
                
            elif init_y<=60:
                mycanvas.showPage()
                page_properties(mycanvas)
                init_x=15
                init_y=650
                mycanvas.drawImage(chart+str(i)+'.png',init_x,init_y,6.4*30,4.1*30)
                
            else:
                continue
            os.remove(chart+str(i)+'.png')
            
    def chart_insertionbiv(data,init_x,init_y,chart):
        for i in range(0,len(data.columns)):
            current_col=data.columns[i]
            if current_col!=target_variable:
                if init_x<=width-120:
                    mycanvas.drawImage(chart+str(i)+'.png',init_x,init_y,6.4*30,4.1*30)
                    init_x+=195

                elif init_y>=195:
                    init_x=15
                    init_y-=135
                    mycanvas.drawImage(chart+str(i)+'.png',init_x,init_y,6.4*30,4.1*30)
                    init_x+=195

                elif init_y<=60:
                    mycanvas.showPage()
                    page_properties(mycanvas)
                    init_x=15
                    init_y=650
                    mycanvas.drawImage(chart+str(i)+'.png',init_x,init_y,6.4*30,4.1*30)

                else:
                    continue
                os.remove(chart+str(i)+'.png')
                
    if combobox=='Analytics':
        filename=entry_output.get()
        output_path=entry_file.get()
        output_path2=stringpath_replace(output_path)
        width,height=letter
        mycanvas=canvas.Canvas(output_path2 + '//'+filename+'.pdf',pagesize=letter)
        document_properties(mycanvas)
        data=dataframe._get_numeric_data()

        for i in range(0,len(data.columns)):
            col=str(data.columns[i-1])
            mean,std=norm.fit(data[col].dropna())
            fig,ax=plt.subplots()
            ax.set_facecolor('#DAEDFA')
            ax.patch.set_facecolor('#DAEDFA')
            ax.hist(data[col].dropna(),60,facecolor='#CC0000',rwidth=0.8)
            ax.set_title(data.columns[i-1]+ ' ' + 'mean=%.1f,std=%.1f' %(mean,std))
            ax.grid(False)
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
            fig.savefig('histogram'+str(i)+'.png',facecolor='#DAEDFA')
            plt.close()


            marker_style=dict(markerfacecolor='#AF2349',marker='h',color='#917C93')
            fig,ax=plt.subplots()
            ax.boxplot(data[col].dropna(),patch_artist=True,flierprops=marker_style,medianprops=dict(color='#840037'),boxprops=dict(color='#CC0000',facecolor='#CC0000'))
            ax.set_facecolor('#DAEDFA')
            ax.patch.set_facecolor('#DAEDFA')
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
            plt.title('Boxplot of'+' '+col)
            ax.grid(False)
            plt.title('Boxplot of'+' '+col)
            fig.savefig('boxplot'+str(i)+'.png',facecolor='#DAEDFA')
            plt.close()

        cat_vars=dataframe.select_dtypes(include=['object'])
        color_plt=[(0.56,0.113,0.192),(0.56,0.113,0.192),(0.56,0.113,0.192),(0.56,0.113,0.192),(0.56,0.113,0.192)]

        for i in range(0,len(cat_vars.columns)):
            col=str(cat_vars.columns[i])

            if len(cat_vars[col].value_counts())>=5:
                plot_cats=cat_vars[col].value_counts().nlargest(5)
                fig,ax=plt.subplots()
                plot_cats.plot.bar(title=col,rot=0,color=color_plt)
                ax.set_facecolor('#FFFFFF')
                ax.patch.set_facecolor('#DAEDFA')
                ax.spines['right'].set_visible(False)
                ax.spines['top'].set_visible(False)
                fig.savefig('univariate'+str(i)+'.png',facecolor='#DAEDFA')
                plt.close()
            else:
                plot_cats=cat_vars[col].value_counts()
                fig,ax=plt.subplots()
                plot_cats.plot.bar(title=col,rot=0,color=color_plt)
                ax.set_facecolor('#FFFFFF')
                ax.patch.set_facecolor('#DAEDFA')
                ax.spines['right'].set_visible(False)
                ax.spines['top'].set_visible(False)
                fig.savefig('univariate'+str(i)+'.png',facecolor='#DAEDFA')
                plt.close()


        x1=pd.DataFrame(index=['Total Observartions','Total Columns','Total Nulls'],data={'Metrics':[str(len(dataframe)),str(len(dataframe.columns)),dataframe.isnull().sum().sum()]})
        x1=x1.reset_index()
        x1.columns.values[0]='Variable'
        x2=df_to_formattedlist(x1)

        summary_numeric=dataframe.describe().transpose().applymap(str)
        summary_numeric3=summary_numeric.applymap(lambda x:x.split('.')[0]).reset_index()
        summary_numeric3=summary_numeric3.applymap(lambda x: x[0:12])
        summary_numeric3.columns.values[0]='Variable'
        x3=df_to_formattedlist(summary_numeric3)
        summary_object=dataframe.describe(include=[np.object]).transpose().applymap(str)
        summary_object2=summary_object.applymap(lambda x:x.split('.')[0]).reset_index()
        summary_object3=summary_object2.applymap(lambda x: x[0:12])
        summary_object3.columns.values[0]='Variable'
        x4=df_to_formattedlist(summary_object3)

        tableb,table1,table2=x2,x3,x4
        textobject0=mycanvas.beginText()
        text_format(textobject0,275,685,12,color=black)
        textobject0.textLines('''Dataset Description''')
        mycanvas.drawText(textobject0)
        t0=Table(tableb,len(tableb[0])*[46],len(tableb)*[20])
        tablestyle(t0)
        t0.wrapOn(mycanvas,9,9)
        t0.drawOn(mycanvas,282,590)

        textobject=mycanvas.beginText()
        text_format(textobject,250,565,12,color=black)
        textobject.textLines('''Continous Variables Description''')
        mycanvas.drawText(textobject)
        set_separator=len(table1)

        if set_separator<25:
            t=Table(table1,colWidths=[45],rowHeights=20)
            tablestyle(t)
            t.wrapOn(mycanvas,9,9)
            x=540-(len(table1)*20)
            t.drawOn(mycanvas,115,x)
            mycanvas.showPage()
            #textobject2=mycanvas.beginText()
            #text_format(textobject2,252,x-20,12)
            #textobject2.textLines('''Categorical Variables Description''')
            #mycanvas.drawText(textobject2)

        else: 
            t=Table(table1[:25],colWidths=[45],rowHeights=20)
            tablestyle(t)
            t.wrapOn(mycanvas,9,9)
            x=540-(len(table1)*20)
            t.drawOn(mycanvas,115,x)
            #textobject5=mycanvas.beginText()
            #text_format(textobject2,252,x-20,12)
            #textobject5.textLines('''Categorical Variables Description''')
            #mycanvas.drawText(textobject5)
            #mycanvas.showPage()
            init=24
            title=['Measure','Count','Mean','Std','Min','Max','25%','50%','75%','Max']
            while(len(table1)-init)/35>=1:
                page_properties(mycanvas)
                t=Table(table1[init:init+35],colWidths=[45],rowHeights=20)
                tablestyle(t)
                x=750-(35*20)
                t0.wrapOn(mycanvas,9,9)
                t0.drawOn(mycanvas,115,x)
                mycanvas.showPage()
                init=init+35

            tableins=table1[init+1:]
            tableins.insert(0,title)
            t=Table(table1,colWidths=[45],rowHeights=20)
            tablestyle(t)
            x=750-(len(table1[init:])*20)
            t0.wrapOn(mycanvas,9,9)
            t0.drawOn(mycanvas,115,x)
            
        page_properties(mycanvas)
        textobject2=mycanvas.beginText()
        text_format(textobject2,235,715,12)
        textobject2.textLines('''Categorical Variables Description''')
        mycanvas.drawText(textobject2)
        t2=Table(table2,colWidths=[45],rowHeights=20)
        tablestyle(t2)
        t2.wrapOn(mycanvas,9,9)
        x2=685-(len(table2)*20)
        t2.drawOn(mycanvas,205,x2)
        mycanvas.showPage()
        page_properties(mycanvas)
        data2=dataframe._get_numeric_data()
        init_x=15
        init_y=600
        textobject=mycanvas.beginText()
        text_format(textobject,0,745,8)
        textobject.textLines(''' Histograms provide a visual interpretation of numerical data by indicating the number of data points that lie within a range of values. These ranges are called classes or bins.
                                     In this case data is represented with 60 bins''')

        mycanvas.drawText(textobject)

        textobject2=mycanvas.beginText()
        text_format(textobject2,290,770,15,color=HexColor('#B31E30'))
        textobject2.textLines('Histogram')
        mycanvas.drawText(textobject2)
        chart_insertion(data=data,init_x=init_x,init_y=init_y,chart='Histogram')
        mycanvas.showPage()
        page_properties(mycanvas)

        init_x=15
        init_y=600
        textobject=mycanvas.beginText()
        text_format(textobject,0,745,8)
        textobject.textLines(''' Boxplot is a method for graphically depicting groups of numerical values through their quantiles. Line in orange represents the median, while highlighted values (red) correspond
                                       to outliers''')
        mycanvas.drawText(textobject)
        textobject2=mycanvas.beginText()
        text_format(textobject2,290,770,15,color=HexColor('#B31E30'))
        textobject2.textLines('Boxplot')
        mycanvas.drawText(textobject2)
        chart_insertion(data=data,init_x=init_x,init_y=init_y,chart='boxplot')
        mycanvas.showPage()
        page_properties(mycanvas)


        data2=dataframe.select_dtypes(include=['object'])
        init_x=15
        init_y=600
        textobject=mycanvas.beginText()
        text_format(textobject,0,745,8)
        textobject.textLines(''' Barchar represent categorical data with rectangular bars with heights and lenghts proportional to the values that they represent . Here we only plot the top 5 categories in
                                     terms of ocurrencies, to check more details about the number of levels, refer to the categorical variables table''')
        mycanvas.drawText(textobject)


        textobject2=mycanvas.beginText()
        text_format(textobject2,290,770,15,color=HexColor('#B31E30'))
        textobject2.textLines('Barchart')
        mycanvas.drawText(textobject2)
        chart_insertion(data=data2,init_x=init_x,init_y=init_y,chart='univariate')
        mycanvas.showPage()
        page_properties(mycanvas)




        #mycanvas.showPage()
        mycanvas.save()
        
    if combobox=='Classification':
        filename=entry_output.get()
        output_path=entry_file.get()
        output_path2=stringpath_replace(output_path)
        width,height=letter
        mycanvas=canvas.Canvas(output_path2 + '//'+filename+'.pdf',pagesize=letter)
        target_variable=e1.get()
        document_properties(mycanvas)
        data=dataframe._get_numeric_data()
        
        numeric_data=dataframe._get_numeric_data()
        numeric_data[target_variable]=dataframe[target_variable].copy()
        summary_numeric=numeric_data.groupby(target_variable).describe(percentiles=[.5]).round(1).transpose().applymap(str)
        summary_numeric=summary_numeric.applymap(lambda x: x[0:12])
        x3=df_to_formattedlist_biv(summary_numeric,dataframe)


        table1=x3
        textobject=mycanvas.beginText()
        x=540-(len(table1)*15)
        y=x+len(table1)*20
        text_format(textobject,226,695,14,color=black)
        textobject.textLines('''Continous Variables Description''')
        mycanvas.drawText(textobject)
        table_header=[['Count','Mean','Std','Min','Median','Max']]
        t=Table(table_header,colWidths=[45*2.0195],rowHeights=20)
        tablestyle(t,fontsize=7)
        t.wrapOn(mycanvas,9,9)
        t.drawOn(mycanvas,54.5,y)
        set_separator=len(table1)

        if set_separator<25:
            t=Table(table1,colWidths=[45*1.01],rowHeights=20)
            tablestyle(t)
            t.wrapOn(mycanvas,9,9)
            x=540-(len(table1)*15)
            t.drawOn(mycanvas,9,x)
            #textobject2=mycanvas.beginText()
            #text_format(textobject2,226,x-30,14)
            #textobject2.textLines('''Categorical Variables Description''')
            #mycanvas.drawText(textobject2)

        else: 
            t=Table(table1[:25],colWidths=[45],rowHeights=20)
            tablestyle(t)
            t.wrapOn(mycanvas,9,9)
            x=540-(len(table1)*20)
            t.drawOn(mycanvas,115,x)
            #textobject5=mycanvas.beginText()
            #text_format(textobject5,252,x-20,12)
            #textobject5.textLines('''Categorical Variables Description''')
            #mycanvas.drawText(textobject5)
            mycanvas.showPage()
            init=24
            while(len(table1)-init)/35>=1:
                page_properties(mycanvas)
                t=Table(table1[init:init+35],colWidths=[45],rowHeights=20)
                tablestyle(t)
                x=750-(35*20)
                t.wrapOn(mycanvas,9,9)
                t.drawOn(mycanvas,115,x)
                mycanvas.showPage()
                init=init+35

            tableins=table1[init+1:]
            tableins.insert(0,title)
            t=Table(table1,colWidths=[45],rowHeights=20)
            tablestyle(t)
            x=750-(len(table1[init:])*20)
            t.wrapOn(mycanvas,9,9)
            t.drawOn(mycanvas,115,x)\
        
        #-----------Insert Bivariate Charts-------#
        numeric_df=dataframe._get_numeric_data()
        mycanvas.showPage()
        page_properties(mycanvas)
        init_x=15
        init_y=600
        textobject=mycanvas.beginText()
        text_format(textobject,0,745,8)
        textobject.textLines(''' Histograms provide a visual interpretation of numerical data by indicating the number of data points that lie within a range of values. These ranges are called classes or bins.
                                     In this case data is represented with 60 bins''')

        mycanvas.drawText(textobject)

        textobject2=mycanvas.beginText()
        text_format(textobject2,290,770,15,color=HexColor('#B31E30'))
        textobject2.textLines('Histogram')
        mycanvas.drawText(textobject2)
        biv_densityplot(numeric_df,target_variable)
        chart_insertionbiv(data=numeric_df,init_x=init_x,init_y=init_y,chart='densitybiv')
        mycanvas.showPage()
        page_properties(mycanvas)
        
        init_x=15
        init_y=600
        textobject=mycanvas.beginText()
        text_format(textobject,0,745,8)
        textobject.textLines(''' Boxplot is a method for graphically depicting groups of numerical values through their quantiles. Line in orange represents the median, while highlighted values (red) correspond
                                       to outliers''')

        mycanvas.drawText(textobject)

        textobject2=mycanvas.beginText()
        text_format(textobject2,290,770,15,color=HexColor('#B31E30'))
        textobject2.textLines('Boxplot')
        mycanvas.drawText(textobject2)
        biv_boxplot(numeric_df,target_variable)
        chart_insertionbiv(data=numeric_df,init_x=init_x,init_y=init_y,chart='boxplotbiv')
        mycanvas.showPage()
        page_properties(mycanvas)
        
        init_x=15
        init_y=600
        textobject=mycanvas.beginText()
        text_format(textobject,0,745,8)
        textobject.textLines(''' Barchar represent categorical data with rectangular bars with heights and lenghts proportional to the values that they represent . Here we only plot the top 5 categories in
                                     terms of ocurrencies, to check more details about the number of levels, refer to the categorical variables table''')

        mycanvas.drawText(textobject)

        textobject2=mycanvas.beginText()
        text_format(textobject2,290,770,15,color=HexColor('#B31E30'))
        textobject2.textLines('Barchart')
        mycanvas.drawText(textobject2)

        biv_barchart(dataframe,target_variable)
        obj_data=dataframe.select_dtypes(include="object")
        obj_data[target_variable] = dataframe[target_variable].apply(str).copy()
        chart_insertionbiv(data=obj_data,init_x=init_x,init_y=init_y,chart='barchartbiv')
        
        
        mycanvas.save()
        
        
eda_window=Tk()
c=Canvas(eda_window,width=500,height=500,bg='#DAEDFB')
c.create_rectangle(0, 480, 500, 500,outline="red", fill="#CC0000")
c.pack()
EDAwindow(eda_window)
eda_window.mainloop()